Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

#CNN tools
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
tf.keras.backend.set_floatx('float64')
#random_state_seed = 0
#tf.random.set_seed(random_state_seed)
#np.random.seed(random_state_seed)

#tools and metrics
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score


Simple data preprocessing without feature selection

In [2]:
df = pd.read_csv('breast-cancer.csv')
                 
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int) 

In [4]:
X_cols = df.columns.to_list()
X_cols.remove('id')
X_cols.remove('diagnosis')
X = df[X_cols]
y = df['diagnosis']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=10)

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)

Creating models

In [7]:
def train_model(model, X_train, y_train, X_test,y_test, model_name):

    model.fit(X_train, y_train)  

    y_pred = model.predict(X_test)

    accuracy            = accuracy_score(y_test, y_pred)
    f1                  = f1_score(y_test, y_pred)
    precision           = precision_score(y_test, y_pred)
    recall              = recall_score(y_test, y_pred)

    result_df = pd.DataFrame([[accuracy, f1, precision, recall]], columns=['accuracy', 'f1_score', 'precision', 'recall'])
    result_df.index = [model_name]
    return result_df

In [8]:
tree = DecisionTreeClassifier(criterion="entropy")

results = train_model(tree, X_train, y_train, X_test, y_test, 'Decision_Tree')

In [9]:
forest_100 = RandomForestClassifier(n_estimators=100,criterion="gini")

results = results.append(train_model(forest_100, X_train, y_train, X_test, y_test, 'Random_Forest')) 

In [10]:
KNN_5 = KNeighborsClassifier(n_neighbors=5)

results = results.append(train_model(KNN_5, X_train, y_train, X_test, y_test, 'KNN_5')) 

In [11]:
KNN_10 = KNeighborsClassifier(n_neighbors=10)

results = results.append(train_model(KNN_10, X_train, y_train, X_test, y_test, 'KNN_10')) 

In [12]:
bayes = GaussianNB()

results = results.append(train_model(bayes, X_train, y_train, X_test, y_test, 'Naive_Bayes')) 

In [13]:
svm = SGDClassifier(loss= "hinge")

results = results.append(train_model(svm, X_train, y_train, X_test, y_test, 'SVM')) 

In [14]:
simple_ann = Sequential()
simple_ann.add(Dense(units=10,activation='relu',))
simple_ann.add(Dense(units=1,activation='sigmoid'))
simple_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
simple_ann.fit(X_train, y_train, batch_size=32, epochs=200)

Epoch 1/200
13/13 [==============================] - 0s 2ms/step - loss: 0.7035 - accuracy: 0.5905
Epoch 2/200
13/13 [==============================] - 0s 769us/step - loss: 0.5867 - accuracy: 0.7261
Epoch 3/200
13/13 [==============================] - 0s 809us/step - loss: 0.4982 - accuracy: 0.8367
Epoch 4/200
13/13 [==============================] - 0s 846us/step - loss: 0.4299 - accuracy: 0.8769
Epoch 5/200
13/13 [==============================] - 0s 769us/step - loss: 0.3760 - accuracy: 0.9045
Epoch 6/200
13/13 [==============================] - 0s 692us/step - loss: 0.3344 - accuracy: 0.9196
Epoch 7/200
13/13 [==============================] - 0s 769us/step - loss: 0.3001 - accuracy: 0.9372
Epoch 8/200
13/13 [==============================] - 0s 769us/step - loss: 0.2724 - accuracy: 0.9372
Epoch 9/200
13/13 [==============================] - 0s 692us/step - loss: 0.2488 - accuracy: 0.9422
Epoch 10/200
13/13 [==============================] - 0s 769us/step - loss: 0.2291 - accuracy

In [15]:
probability_pred = simple_ann.predict(X_test)
y_pred = (probability_pred > 0.4)
accuracy =          accuracy_score(y_test, y_pred)
f1 =                f1_score(y_test, y_pred)
precision =         precision_score(y_test, y_pred)
recall =            recall_score(y_test, y_pred)
simple_ann_result = pd.DataFrame([[accuracy, f1, precision, recall]], columns=['accuracy', 'f1_score', 'precision', 'recall'])
simple_ann_result.index = ['Simple_Neural_Network']
results = results.append(simple_ann_result)

In [16]:
complex_ann = Sequential()
complex_ann.add(Dense(units=32,activation='relu'))
complex_ann.add(Dense(units=8,activation='relu'))
complex_ann.add(Dense(units=1,activation='sigmoid'))
complex_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
complex_ann.fit(X_train, y_train, batch_size=32, epochs=200)

Epoch 1/200
13/13 [==============================] - 0s 854us/step - loss: 0.7301 - accuracy: 0.4221
Epoch 2/200
13/13 [==============================] - 0s 820us/step - loss: 0.5924 - accuracy: 0.5930
Epoch 3/200
13/13 [==============================] - 0s 988us/step - loss: 0.5234 - accuracy: 0.7688
Epoch 4/200
13/13 [==============================] - 0s 769us/step - loss: 0.4708 - accuracy: 0.8744
Epoch 5/200
13/13 [==============================] - 0s 845us/step - loss: 0.4230 - accuracy: 0.9196
Epoch 6/200
13/13 [==============================] - 0s 769us/step - loss: 0.3742 - accuracy: 0.9397
Epoch 7/200
13/13 [==============================] - 0s 846us/step - loss: 0.3269 - accuracy: 0.9497
Epoch 8/200
13/13 [==============================] - 0s 769us/step - loss: 0.2822 - accuracy: 0.9598
Epoch 9/200
13/13 [==============================] - 0s 769us/step - loss: 0.2400 - accuracy: 0.9598
Epoch 10/200
13/13 [==============================] - 0s 769us/step - loss: 0.2018 - accura

In [17]:
probability_pred = complex_ann.predict(X_test)
y_pred = (probability_pred > 0.4)
accuracy =          accuracy_score(y_test, y_pred)
f1 =                f1_score(y_test, y_pred)
precision =         precision_score(y_test, y_pred)
recall =            recall_score(y_test, y_pred)
complex_ann_result = pd.DataFrame([[accuracy, f1, precision, recall]], columns=['accuracy', 'f1_score', 'precision', 'recall'])
complex_ann_result.index = ['Complex_Neural_Network']
results = results.append(complex_ann_result)

In [18]:
results.style.background_gradient(cmap = sns.color_palette("flare", as_cmap=True))

,accuracy,f1_score,precision,recall
Decision_Tree,0.941520,0.918033,0.888889,0.949153
Random_Forest,0.982456,0.975207,0.951613,1.000000
KNN_5,0.982456,0.974359,0.982759,0.966102
KNN_10,0.964912,0.947368,0.981818,0.915254
Naive_Bayes,0.947368,0.925620,0.903226,0.949153
SVM,0.935673,0.910569,0.875000,0.949153
Simple_Neural_Network,0.982456,0.974790,0.966667,0.983051
Complex_Neural_Network,0.982456,0.974790,0.966667,0.983051


Second attempt with more data preprocessing, with feature selection based on correlation

In [19]:
df = pd.read_csv('breast-cancer.csv')

In [20]:
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int) 
df = df.drop('id',axis=1)

In [21]:
correlations = df.corr()['diagnosis']
correlations = correlations.drop('diagnosis',axis=0).rename('correlation with diagnosis')
abs_correlations = abs(correlations.rename('abs correlation with diagnosis'))
all_correlations = pd.concat([correlations,abs_correlations],axis=1)
pd.set_option("display.max_rows", 40)
all_correlations.sort_values(by='abs correlation with diagnosis', ascending=False)


,correlation with diagnosis,abs correlation with diagnosis
concave points_worst,0.793566,0.793566
perimeter_worst,0.782914,0.782914
concave points_mean,0.776614,0.776614
radius_worst,0.776454,0.776454
perimeter_mean,0.742636,0.742636
area_worst,0.733825,0.733825
radius_mean,0.730029,0.730029
area_mean,0.708984,0.708984
concavity_mean,0.696360,0.696360
concavity_worst,0.659610,0.659610


Choosing most relevent features

In [22]:
relevant_features = abs_correlations[abs_correlations>0.1] 
relevant_names = [index for index, _ in relevant_features.iteritems()]

In [23]:
X = df[relevant_names]
y = df['diagnosis']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=10)


In [25]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
tree = DecisionTreeClassifier(criterion="entropy")

results2 = train_model(tree, X_train, y_train, X_test, y_test, 'Decision_Tree')

In [27]:
forest_100 = RandomForestClassifier(n_estimators=100,criterion="gini")

results2 = results2.append(train_model(forest_100, X_train, y_train, X_test, y_test, 'Random_Forest')) 

In [28]:
KNN_5 = KNeighborsClassifier(n_neighbors=5)

results2 = results2.append(train_model(KNN_5, X_train, y_train, X_test, y_test, 'KNN_5')) 

In [29]:
KNN_10 = KNeighborsClassifier(n_neighbors=10)

results2 = results2.append(train_model(KNN_10, X_train, y_train, X_test, y_test, 'KNN_10')) 

In [30]:
bayes = GaussianNB()

results2 = results2.append(train_model(bayes, X_train, y_train, X_test, y_test, 'Naive_Bayes')) 

In [31]:
svm = SGDClassifier(loss= "hinge")

results2 = results2.append(train_model(svm, X_train, y_train, X_test, y_test, 'SVM')) 

In [32]:
simple_ann = Sequential()
simple_ann.add(Dense(units=10,activation='relu',))
simple_ann.add(Dense(units=1,activation='sigmoid'))
simple_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
simple_ann.fit(X_train, y_train, batch_size=32, epochs=200)

probability_pred = simple_ann.predict(X_test)
y_pred = (probability_pred > 0.4)
accuracy =          accuracy_score(y_test, y_pred)
f1 =                f1_score(y_test, y_pred)
precision =         precision_score(y_test, y_pred)
recall =            recall_score(y_test, y_pred)
simple_ann_result = pd.DataFrame([[accuracy, f1, precision, recall]], columns=['accuracy', 'f1_score', 'precision', 'recall'])
simple_ann_result.index = ['Simple_Neural_Network']
results2 = results2.append(simple_ann_result)

Epoch 1/200
13/13 [==============================] - 0s 1000us/step - loss: 0.7682 - accuracy: 0.5704
Epoch 2/200
13/13 [==============================] - 0s 923us/step - loss: 0.6491 - accuracy: 0.6432
Epoch 3/200
13/13 [==============================] - 0s 1000us/step - loss: 0.5561 - accuracy: 0.7010
Epoch 4/200
13/13 [==============================] - 0s 922us/step - loss: 0.4855 - accuracy: 0.7688
Epoch 5/200
13/13 [==============================] - 0s 922us/step - loss: 0.4307 - accuracy: 0.8317
Epoch 6/200
13/13 [==============================] - 0s 769us/step - loss: 0.3889 - accuracy: 0.8643
Epoch 7/200
13/13 [==============================] - 0s 692us/step - loss: 0.3552 - accuracy: 0.8844
Epoch 8/200
13/13 [==============================] - 0s 692us/step - loss: 0.3273 - accuracy: 0.9020
Epoch 9/200
13/13 [==============================] - 0s 923us/step - loss: 0.3019 - accuracy: 0.9095
Epoch 10/200
13/13 [==============================] - 0s 923us/step - loss: 0.2802 - accu

In [33]:
complex_ann = Sequential()
complex_ann.add(Dense(units=32,activation='relu'))
complex_ann.add(Dense(units=8,activation='relu'))
complex_ann.add(Dense(units=1,activation='sigmoid'))
complex_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
complex_ann.fit(X_train, y_train, batch_size=32, epochs=200)

probability_pred = complex_ann.predict(X_test)
y_pred = (probability_pred > 0.4)
accuracy =          accuracy_score(y_test, y_pred)
f1 =                f1_score(y_test, y_pred)
precision =         precision_score(y_test, y_pred)
recall =            recall_score(y_test, y_pred)
complex_ann_result = pd.DataFrame([[accuracy, f1, precision, recall]], columns=['accuracy', 'f1_score', 'precision', 'recall'])
complex_ann_result.index = ['Complex_Neural_Network']
results2 = results2.append(complex_ann_result)

Epoch 1/200
13/13 [==============================] - 0s 1ms/step - loss: 0.5585 - accuracy: 0.6608
Epoch 2/200
13/13 [==============================] - 0s 1ms/step - loss: 0.4316 - accuracy: 0.8492
Epoch 3/200
13/13 [==============================] - 0s 1ms/step - loss: 0.3370 - accuracy: 0.9196
Epoch 4/200
13/13 [==============================] - 0s 2ms/step - loss: 0.2681 - accuracy: 0.9322
Epoch 5/200
13/13 [==============================] - 0s 1ms/step - loss: 0.2201 - accuracy: 0.9347
Epoch 6/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1869 - accuracy: 0.9372
Epoch 7/200
13/13 [==============================] - 0s 2ms/step - loss: 0.1614 - accuracy: 0.9523
Epoch 8/200
13/13 [==============================] - 0s 1ms/step - loss: 0.1427 - accuracy: 0.9623
Epoch 9/200
13/13 [==============================] - 0s 1ms/step - loss: 0.1277 - accuracy: 0.9724
Epoch 10/200
13/13 [==============================] - ETA: 0s - loss: 0.0524 - accuracy: 1.00 - 0s 1ms/step -

In [34]:
results2.style.background_gradient(cmap = sns.color_palette("flare", as_cmap=True))

,accuracy,f1_score,precision,recall
Decision_Tree,0.947368,0.925620,0.903226,0.949153
Random_Forest,0.982456,0.975207,0.951613,1.000000
KNN_5,0.988304,0.983333,0.967213,1.000000
KNN_10,0.964912,0.947368,0.981818,0.915254
Naive_Bayes,0.941520,0.916667,0.901639,0.932203
SVM,0.953216,0.935484,0.892308,0.983051
Simple_Neural_Network,0.976608,0.966667,0.950820,0.983051
Complex_Neural_Network,0.982456,0.974790,0.966667,0.983051


Use **Sequential Feature Selection** for choose best features for KNN model 

In [35]:
df = pd.read_csv('breast-cancer.csv')
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int) 
df = df.drop('id',axis=1)
X_cols = df.columns.to_list()
X_cols.remove('diagnosis')
X = df[X_cols]
y = df['diagnosis']
scaler = StandardScaler() 
scaler.fit(X) 
X.iloc[:,:] = scaler.fit_transform(X.iloc[:,:])

sfs = SFS(KNeighborsClassifier(n_neighbors=5),
           k_features=30,
           forward=True,
           floating=False,
           scoring = 'recall',
           cv = 0)

sfs.fit(X, y)

df_SFS_results = pd.DataFrame(sfs.subsets_).transpose()
df_SFS_results


d:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
d:\Anaconda\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,feature_idx,cv_scores,avg_score,feature_names
1,"(7,)",[0.8867924528301887],0.886792,"(concave points_mean,)"
2,"(7, 23)",[0.9292452830188679],0.929245,"(concave points_mean, area_worst)"
3,"(6, 7, 23)",[0.9669811320754716],0.966981,"(concavity_mean, concave points_mean, area_worst)"
4,"(6, 7, 20, 23)",[0.9716981132075472],0.971698,"(concavity_mean, concave points_mean, radius_w..."
5,"(6, 7, 17, 20, 23)",[0.9669811320754716],0.966981,"(concavity_mean, concave points_mean, concave ..."
6,"(6, 7, 17, 20, 23, 27)",[0.9622641509433962],0.962264,"(concavity_mean, concave points_mean, concave ..."
7,"(6, 7, 8, 17, 20, 23, 27)",[0.9622641509433962],0.962264,"(concavity_mean, concave points_mean, symmetry..."
8,"(6, 7, 8, 15, 17, 20, 23, 27)",[0.9622641509433962],0.962264,"(concavity_mean, concave points_mean, symmetry..."
9,"(6, 7, 8, 12, 15, 17, 20, 23, 27)",[0.9575471698113207],0.957547,"(concavity_mean, concave points_mean, symmetry..."
10,"(6, 7, 8, 12, 14, 15, 17, 20, 23, 27)",[0.9622641509433962],0.962264,"(concavity_mean, concave points_mean, symmetry..."


In [36]:
relevant_names = list(df_SFS_results['feature_names'][1])
X_rel = df[relevant_names]
y_rel = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X_rel, y_rel, test_size = 0.3,random_state=10)
scaler = StandardScaler() 
scaler.fit(X_train) 

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 
KNN = KNeighborsClassifier(n_neighbors=5)
results3 = train_model(KNN, X_train, y_train, X_test, y_test, '1')


for i in range(2,31):
    relevant_names = list(df_SFS_results['feature_names'][i])
    X_rel = df[relevant_names]
    y_rel = df['diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X_rel, y_rel, test_size = 0.3,random_state=30)
    scaler = StandardScaler() 
    scaler.fit(X_train) 

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test) 
    KNN = KNeighborsClassifier(n_neighbors=5)
    results3 = results3.append(train_model(KNN, X_train, y_train, X_test, y_test, str(i)))

results3.style.background_gradient(cmap = sns.color_palette("flare", as_cmap=True))

,accuracy,f1_score,precision,recall
1,0.935673,0.905983,0.913793,0.898305
2,0.941520,0.919355,0.919355,0.919355
3,0.923977,0.896000,0.888889,0.903226
4,0.935673,0.912000,0.904762,0.919355
5,0.953216,0.935484,0.935484,0.935484
6,0.941520,0.916667,0.948276,0.887097
7,0.970760,0.957983,1.000000,0.919355
8,0.970760,0.957983,1.000000,0.919355
9,0.970760,0.957983,1.000000,0.919355
10,0.964912,0.950000,0.982759,0.919355


Show order how features were selected

In [37]:
rank = [df_SFS_results['feature_names'][1][0]]
for i in range(2,31):
    rank.append((list(set(df_SFS_results['feature_names'][i]) - set(df_SFS_results['feature_names'][i-1]))[0]))

pd.DataFrame(rank,columns=['feature'],index=[i for i in range(1,31)])

,feature
1,concave points_mean
2,area_worst
3,concavity_mean
4,radius_worst
5,concave points_se
6,concave points_worst
7,symmetry_mean
8,compactness_se
9,perimeter_se
10,smoothness_se


Use **Sequential Feature Eimination** for choose best features for KNN model 

In [38]:
df = pd.read_csv('breast-cancer.csv')
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int) 
df = df.drop('id',axis=1)
X_cols = df.columns.to_list()
X_cols.remove('diagnosis')
X = df[X_cols]
y = df['diagnosis']
scaler = StandardScaler() 
scaler.fit(X) 
X.iloc[:,:] = scaler.fit_transform(X.iloc[:,:])

sfs = SFS(KNeighborsClassifier(n_neighbors=5),
           k_features=1,
           forward=False,
           floating=False,
           scoring = 'recall',
           cv = 0)

sfs.fit(X, y)

df_SFS_results = pd.DataFrame(sfs.subsets_).transpose()
df_SFS_results

d:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
d:\Anaconda\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,feature_idx,cv_scores,avg_score,feature_names
30,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[0.9528301886792453],0.95283,"(radius_mean, texture_mean, perimeter_mean, ar..."
29,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[0.9575471698113207],0.957547,"(radius_mean, texture_mean, perimeter_mean, ar..."
28,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,...",[0.9575471698113207],0.957547,"(radius_mean, texture_mean, perimeter_mean, ar..."
27,"(0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15...",[0.9575471698113207],0.957547,"(radius_mean, texture_mean, perimeter_mean, ar..."
26,"(0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 14, 15, 1...",[0.9575471698113207],0.957547,"(radius_mean, texture_mean, perimeter_mean, ar..."
25,"(0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 14, 15, 1...",[0.9528301886792453],0.95283,"(radius_mean, texture_mean, perimeter_mean, ar..."
24,"(0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 14, 15, 1...",[0.9575471698113207],0.957547,"(radius_mean, texture_mean, perimeter_mean, ar..."
23,"(0, 1, 2, 3, 4, 6, 8, 9, 10, 12, 14, 15, 16, 1...",[0.9669811320754716],0.966981,"(radius_mean, texture_mean, perimeter_mean, ar..."
22,"(0, 1, 2, 3, 4, 6, 8, 9, 10, 12, 14, 15, 16, 1...",[0.9716981132075472],0.971698,"(radius_mean, texture_mean, perimeter_mean, ar..."
21,"(0, 1, 2, 3, 4, 6, 8, 9, 10, 12, 14, 15, 16, 1...",[0.9716981132075472],0.971698,"(radius_mean, texture_mean, perimeter_mean, ar..."


In [39]:
relevant_names = list(df_SFS_results['feature_names'][30])
X_rel = df[relevant_names]
y_rel = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X_rel, y_rel, test_size = 0.3,random_state=10)
scaler = StandardScaler() 
scaler.fit(X_train) 

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 
KNN = KNeighborsClassifier(n_neighbors=5)
results4 = train_model(KNN, X_train, y_train, X_test, y_test, '30')


for i in range(29,0,-1):
    relevant_names = list(df_SFS_results['feature_names'][i])
    X_rel = df[relevant_names]
    y_rel = df['diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X_rel, y_rel, test_size = 0.3,random_state=10)
    scaler = StandardScaler()
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    KNN = KNeighborsClassifier(n_neighbors=5)
    results4 = results4.append(train_model(KNN, X_train, y_train, X_test, y_test, str(i)))

results4.style.background_gradient(cmap = sns.color_palette("flare", as_cmap=True))

,accuracy,f1_score,precision,recall
30,0.982456,0.974359,0.982759,0.966102
29,0.982456,0.974359,0.982759,0.966102
28,0.982456,0.974359,0.982759,0.966102
27,0.982456,0.974359,0.982759,0.966102
26,0.976608,0.965517,0.982456,0.949153
25,0.976608,0.965517,0.982456,0.949153
24,0.970760,0.956522,0.982143,0.932203
23,0.970760,0.957265,0.965517,0.949153
22,0.964912,0.948276,0.964912,0.932203
21,0.964912,0.948276,0.964912,0.932203


Show order how features were rejected

In [40]:
rank = [df_SFS_results['feature_names'][1][0]]
for i in range(2,31):
    rank.append((list(set(df_SFS_results['feature_names'][i]) - set(df_SFS_results['feature_names'][i-1]))[0]))

pd.DataFrame(rank,columns=['feature'],index=[i for i in range(1,31)]).sort_index(ascending=False)

,feature
30,fractal_dimension_se
29,area_se
28,concave points_mean
27,compactness_mean
26,symmetry_worst
25,concave points_worst
24,texture_se
23,concave points_se
22,perimeter_worst
21,fractal_dimension_worst
